# 1. Datenimport und -bereinigung
---
## 1.1 Datenimport
---

Hier werden aus einem csv file Pflanzendaten (Insektenfreundliche, Standortansprüche, Blütemonate etc.) zur Berechnung des Bienen- und des Blühzeitscores in ein pandas Dataframe gelesen.

### 1.1.1 Importieren der benötigten Libraries:
- _pandas_
- _pandas profiling_

In [ ]:
import pandas as pd
import pandas_profiling as pp

### 1.1.2 Einlesen der csv-Datei "DatenBalkonien" aus dem Importordner und erste Sichtung der Daten

In [ ]:
#Import der Daten als Dataframe
plantfacts_df = pd.read_csv('./Import/DatenBalkonien.csv',sep=",")

In [ ]:
#Anzeigen Typ und Prüfung, ob Pandas Dataframe erstellt wurde.
print(type(plantfacts_df))

In [ ]:
#Anzeigen Attribute
plantfacts_df.head()

In [ ]:
#Anzeigen Dataframe und Durchsicht auf Null-Werte / Richtigkeit Datentypen.
plantfacts_df.info()

## 1.2 Datenbereinigung 
---

Es wird eine Kopie des Dataframes erstellt, die um nicht genutzte Attribute sowie Duplikate und fehlerhafte Werte (hier: nicht vorhanden) bereinigt wird. 
Spaltenbezeichnungen und Pflanzennamen werden vereinheitlicht (lowercase, Spaltenbezeichnungen: keine Leerzeichen).

In [ ]:
#Kopie des Dataframes und Aussortieren der Attribute (Spalten), die nicht weiter verwendet werden.
plants_df = plantfacts_df.copy()
plants_df. drop(plants_df.columns[[1,4,5,6,7,8,9,10,11,12,13,26]],
                    axis = 1,
                    inplace = True)

In [ ]:
#Ausgabe des neuen Dataframes.
plants_df.info()

In [ ]:
#Duplikate finden und beheben. 
pp.ProfileReport(plants_df, minimal=True)

In [ ]:
#Es gibt einige Duplikate, z.B. Oregano:
print(plants_df.loc[plants_df['Name'] == 'Oregano'])


In [ ]:
#Löschen von Duplikaten in der Spalte 'Namen' aus dem Dataframe, da dieser für den Abgleich mit der Usereingabe benötigt wird:
plants_df.drop_duplicates(subset=['Name'], keep='first', inplace=True, ignore_index=True)
print(plants_df.loc[plants_df['Name'] == 'Oregano']) 

In [ ]:
#Spaltenbezeichnungen: Lowercase und _ statt Leerzeichen.
plants_df = plants_df.rename(str.lower, axis='columns')

#Pflanzennamen: Lowercase.
plants_df['name'] = plants_df['name'].str.lower()
plants_df['kategorie'] = plants_df['kategorie'].str.lower()
plants_df.head()

**Anmerkung: Befehl lower für ganzes Dataframe wäre besser**

In [ ]:
#Ausgeben des Dataframes.
plants_df.info()
#Ausgeben als csv file.
plants_df.to_csv('./Import/plants_df.csv', index=False)

# 2. Programm
---

Es werden aufgrund einer Usereingabe (2.1) der auf Balkon / im Garten befindlichen Pflanzen der Biodiversitätsindex (2.2) und ein Bienenscore berechnet sowie im Anschluss eine Empfehlung zur Verbesserung des Bienenscores (2.3) gegeben. Es wird ebenfalls angezeigt, in welchen Monaten keine Pflanze blüht und somit keine Nahrung für Bienen geboten wird (2.4). 

## 2.1 Usereingabe Pflanzen
---

Mit Hilfe einer Funktion für die Usereingabe wird abgefragt, wie viele unterschiedliche Pflanzenarten auf dem Balkon/im Garten des Users wachsen, welche das sind und wie viele Pflanzen jeweils.
Die Eingaben werden in ein Dataframe user_input() geschrieben.

Das Programm für die Eingabe enthält ebenfalls Checks der Eingabedaten zur Vermeidung fehlerhafter Eingaben. Diese Checks sind als separate Fuktionen geschieben.

*Beispiele für mögliche Usereingabe*

*positives Beispiel (8): Kornelkirsche, Oregano, Rose ungefüllt, gewöhnliche Waldrebe, Safran, Tulpe, Kornblume, Sonnenblume (jeweils 5)*

*negatives Beispiel (3): Bauernhortensie (50), Berg-Waldrebe (1), Wald-Erdbeere(1)*

In [ ]:
# Check, ob die eingebenen Pflanzenart im Dataframe der Pflanzen überhaupt vorhanden ist 
# sowie ebefalls, ob die Pflanzenart bereits eingegeben wurde 
def input_plant_name(plant):
    ad = input("Name der Pflanzenart: ")
    
    # check 1: in Liste?
    if ad.lower() not in plants_df["name"].tolist():
        print("Pflanzenart ist nicht bekannt. Bitte erneut eingeben.")
        ad = input_plant_name(plant)
        
    # check 2: Eingabe bereits erfolgt
    if ad.lower() in plant:
        print("Du hast diese Pflanzenart bereits eingegeben. Bitte eine andere Art eingeben.")
        ad = input_plant_name(plant)
    return ad

# Check, ob die eingegeben Anzahl von Pflanzen ein Integer ist
def check_plant_number(input):
    try:
        # Convert it into integer
        val = int(input)
        return val
    except ValueError:
        print("Anzahl ist keine Zahl. Bitte erneut eingeben.")
        return input_plant_number()
    return val


def input_plant_number():
    return check_plant_number(input("Anzahl: ")) 
    
# Schleife für die Eingabe der Pflanzenarten und deren Namen sowie Anzahl
def user_input():
    n = int(input("Wie viele Pflanzenarten haben Sie insgesamt?: "))
    plant = []
    plant_number = []
    for i in range(n):
        ad = input_plant_name(plant)
        nm = input_plant_number()
        ad_l = ad.lower()
        plant.append(ad_l)
        plant_number.append(nm)
    list_user = {"name": plant, "anzahl": plant_number}
    user_df = pd.DataFrame(list_user)
    return user_df


user_df = user_input() 
user_df

*Beispiele für mögliche Usereingabe*

*1. Beispiel (positiv) (8): Kornelkirsche, Oregano, Rose ungefüllt, gewöhnliche Waldrebe, Safran, Tulpe, Kornblume, Sonnenblume (jeweils 5)*

*2. Beispiel (negativ) (3): Bauernhortensie (50), Berg-Waldrebe (1), Wald-Erdbeere(1)*

## 2.2 Diversitätsberechnung
---

Hier wird die Biodiversität anhand des XYZ Index berechnet... Der Index eignet sich für .. Ein Index < ... ist gut, ein Index > zu gering...

Der Shannon_index ist einer der häufigst angewandten Index für die Biodiversität. Dieser eignet sich insbesonderes für klar abgegränzte Lebensräume (Nagendra 2022). In der Natur liegt der Shannon-Indes zwischen 1.5 und 3.5 (Suspense et al. 2016).
"Weil der Diversitätsindex alleine nicht erkennen lässt, ob sein Wert aufgrund einer hohen Artenzahl mit jeweils unterschiedlicher Individuenzahl oder durch gleichmäßige Verteilung der Individuen auf wenige Arten entstanden ist, benutzt man als Vergleichsmaß die berechnete Evenness. Man setzt dabei den tatsächlichen Diversitätswert in Relation zu dem maximal möglichen Diversitätswert, der sich bei gleicher Artenzahl, aber unter größtmöglicher Gleichverteilung der Individuen auf die bestehenden Arten ergeben würde. Der Wert für die Evenness liegt dabei zwischen 0 (völlig ungleiche Verteilung der Individuen auf die einzelnen Arten) und 1 (totale Gleichverteilung)" (Spektrum 2022).

In [ ]:
#ANPASSEN: Berechnung der Alpha Diversität (in einem abgrenzbaren Lebensraum) mit Shannon-Index (H):

from math import log as ln

def shannon_index(user_input):
    def p(n, N):
         return (float(n) / N) * ln(float(n) / N)
    N = user_input.anzahl.sum()

    sdi = -sum(p(n, N) for n in user_input.anzahl if n != 0)
    
    #Diversitätswert in Relation zu dem maximal möglichen Diversitätswert
    evenn = sdi / ln(user_input.name.count())
    
    
    print("Dein Shannon-Index (H) ist:", round(sdi,4))
    print("Die Evenness (w) beträgt:", round(evenn,4))
    
    
    if evenn > 0.8 or sdi > 2:
        print("Dein Balkon ist ausgeglichen bepflanzt und hat eine gute Biodiversität")
    else:
        print("Dein Balkon hat eine geringe Biodiversität. Es scheint, dass eine oder wenige Pflanze verhältnissmässig oft vorkommen.")
        
shannon_index(user_df)

**Anmerkung Anna:**

**Ich bekomme auch einen guten BioDiv, wenn ich nur 2 verschiede Pflanzenarten mit je 2 Pflanzen eingebe. 
Ab wann kann man das Ergebnis als gut bezeichnen? 
Nach diesem Video (https://www.youtube.com/watch?v=-FsRYC5QwkA ca. min 3:05) ist 
    < 1 eine sehr niedriger Alpha Diversität, 
    bis 1,5 vergleichsweise niedrige Diversität
    ab > 2 hohe Diversität.**

## 2.3 Bienenscore
---

Hier wird aus der Anzahl bienenfreundlicher Pflanzen (bienenfreundliche Pflanze * Anzahl) ein Bienescore (Anzahl bienenfreundliche Pflanzen / Pflanzen gesamt) berechnet. Ist dieser < 85%, wird eine Empfehlung ausgegeben, mit welchen bienenfreundlichen Pflanzen der Score verbessert werden kann.

In [ ]:
#neues Dataframe mit allen Spalten aus plants_df zu Usereingabe erstellen.
user_plants_df = pd.merge(plants_df, user_df, on=['name'], how='inner')

#Spalte mit Score für für Bienen wertvolle Pflanzen.
user_plants_df = user_plants_df.assign(score_biene=user_plants_df['biene']*user_plants_df['anzahl'])
user_plants_df

In [ ]:
#Zeile mit Summe je Spalte hinzufügen oder Summe score_biene berechnen.
Total = user_plants_df['score_biene'].sum()/user_df['anzahl'].sum()*100

print("Anzahl bienenfreundliche Pflanzen: ",user_plants_df['score_biene'].sum())
print("Anzahl Pflanzen gesamt: ", user_df['anzahl'].sum())
print("Dein Bienenscore ist =",Total,"%.")

In [ ]:
#Feedback zu Bienenscore:
if Total > 85:
    print("Das ist super! Glückwunsch :-)")
else:
    print("Du solltest mehr bienenfreundliche Pflanzen pflanzen. Zum Beispiel:")
if Total <= 85:
    subset_biene_df = plants_df.loc[plants_df['biene'] == 1]
    #Empfehlung, falls Bienenscore <=85%:
    print(subset_biene_df.name.sample(5))

## 2.4 Blühzeitscore
---

Hier wird dargestellt, in welchen Monaten die vorhandenen Pflanzen Blüte tragen und welche davon für Bienen nützlich sind. Fehlen Blüten für Bienen in Monaten außer Januar und Februar wird der User darauf hingewiesen beim nächsten Pflanzenkauf darauf zu achten, bienenfreundliche Pflanzen mit Blüte in diesen Monaten zu kaufen.

In [ ]:
#Dataframe über Monat, Blüte und bienenfreundliche Blüte:
monat = ['jan', 'feb', 'mrz', 'apr', 'mai', 'jun', 'jul', 'aug', 'sep', 'okt', 'nov', 'dez'] 
blüte = list(map(lambda monat: user_plants_df[monat].sum(), monat))
subset_user_plants_df=user_plants_df.loc[user_plants_df['biene'] == 1]
blüte_biene = list(map(lambda monat: subset_user_plants_df[monat].sum(), monat))

blüte_anz_monat_df = pd.DataFrame({'monat': monat, 'blüte': blüte, 'blüte_biene': blüte_biene})

#Visualisierung der Anzahl blühender Pflanzenarten sowie bienenfreundlicher blühenden Pflanzenarten je Monat
ax_blüte = blüte_anz_monat_df.plot.bar(x='monat', rot=0)

In [ ]:
#Hinweis, in welchen Monaten Blüte für Bienen fehlt. 
keine_blüte_biene = blüte_anz_monat_df.loc[blüte_anz_monat_df['blüte_biene'] == 0]

from functools import reduce
monate = reduce(lambda a, b: a + ', ' + b, keine_blüte_biene['monat'].tolist())

if monate == 'jan, dez':
    print("Super!")
else:
    print("In den folgenden Monaten blüht keine Pflanze:", monate)
    print("Du solltest beim nächsten Pflanzenkauf darauf achten Pflanzen zu wählen, die bienenfreundlich sind und in diesen Monaten blühen.")

In [ ]:
#Empfehlung, in welchen Monaten noch Blüte fehlt, ggf. welche Pflanzen das ändern könnten. 